In [2]:
from pyspark.sql import SparkSession, functions as F, types as T
from pyspark import SparkConf, SparkContext
import requests
import os
import tempfile
from dotenv import load_dotenv



In [3]:
conf = SparkConf()
conf.set("spark.jars.packages", "net.snowflake:snowflake-jdbc:3.24.2,net.snowflake:spark-snowflake_2.12:3.1.2")
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()

In [4]:
load_dotenv(dotenv_path="/home/jovyan/work/.env")

sfOptions = {
    "sfURL": os.getenv("URL"),
    "sfDatabase": os.getenv("DB"),
    "sfSchema": "RAW",
    "sfWarehouse": os.getenv("WAREHOUSE"),
    "sfRole": os.getenv("ROLE"),
    "sfUser": os.getenv("USER"),
    "sfPassword": os.getenv("PASSWORD")
}


In [ ]:

years = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
services = ['green', 'yellow'] #agregar 'yellow' si se desea procesar también

for service in services:
    if service == 'yellow':
        ordered_cols = [
        "VENDORID", "TPEP_PICKUP_DATETIME", "TPEP_DROPOFF_DATETIME",
        "PASSENGER_COUNT", "TRIP_DISTANCE", "RATECODEID",
        "STORE_AND_FWD_FLAG", "PULOCATIONID", "DOLOCATIONID",
        "PAYMENT_TYPE", "FARE_AMOUNT", "EXTRA", "MTA_TAX",
        "TIP_AMOUNT", "TOLLS_AMOUNT", "IMPROVEMENT_SURCHARGE",
        "TOTAL_AMOUNT", "CONGESTION_SURCHARGE", "AIRPORT_FEE",
        "CBD_CONGESTION_FEE",  # ¡asegúrate de este orden exacto!
        "RUN_ID", "SERVICE_TYPE", "SOURCE_YEAR", "SOURCE_MONTH",
        "INGESTED_AT_UTC", "SOURCE_PATH"
        ]
    else:
        ordered_cols = [
        "VENDORID", "LPEP_PICKUP_DATETIME", "LPEP_DROPOFF_DATETIME",
        "STORE_AND_FWD_FLAG", "RATECODEID",
        "PULOCATIONID", "DOLOCATIONID", "PASSENGER_COUNT", "TRIP_DISTANCE",
        "FARE_AMOUNT", "EXTRA", "MTA_TAX", "TIP_AMOUNT", "TOLLS_AMOUNT",
        "EHAIL_FEE", "IMPROVEMENT_SURCHARGE", "TOTAL_AMOUNT",
        "PAYMENT_TYPE", "TRIP_TYPE", "CONGESTION_SURCHARGE",
        "CBD_CONGESTION_FEE",  
        "RUN_ID", "SERVICE_TYPE", "SOURCE_YEAR", "SOURCE_MONTH",
        "INGESTED_AT_UTC", "SOURCE_PATH"
        ]
    for year in years:
        if year == '2025':
            months = ['01', '02', '03', '04', '05', '06', '07']
        for month in months:
            if service == 'yellow':
                timestamp_columns = ['TPEP_PICKUP_DATETIME', 'TPEP_DROPOFF_DATETIME']
            else:
                timestamp_columns = ['LPEP_PICKUP_DATETIME', 'LPEP_DROPOFF_DATETIME']


            url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{service}_tripdata_{year}-{month}.parquet"
            response = requests.head(url)
            if response.status_code != 200:
                print(f"No existe: {url}")
                continue
            print(f"Procesando {service}: {year}-{month}")

            # Descargar archivo temporalmente
            tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".parquet")
            tmp_path = tmp_file.name
            tmp_file.close()
            r = requests.get(url)
            with open(tmp_path, "wb") as f:
                f.write(r.content)

            # Leer archivo con Spark
            df = spark.read.parquet(tmp_path)
            print(f"Archivo {url} cargado en DataFrame")

            # Crear run_id para indempotencia
            run_id = f"run_{year}_{month}"

            # Agregar metadatos
            df = (
                df.withColumn("RUN_ID", F.lit(run_id))
                  .withColumn("SERVICE_TYPE", F.lit(service))
                  .withColumn("SOURCE_YEAR", F.lit(int(year)))
                  .withColumn("SOURCE_MONTH", F.lit(int(month)))
                  .withColumn("INGESTED_AT_UTC", F.to_utc_timestamp(F.current_timestamp(), "America/Guayaquil")) #En UTC
                  .withColumn("SOURCE_PATH", F.lit(url))
            )

            # Normalizar nombres de columnas a mayusculas
            df = df.toDF(*[c.upper() for c in df.columns])
            print("Metadatos agregados")

            # Convertir columnas de timestamp
            for col_name in timestamp_columns:
                if col_name in df.columns:
                    df = df.withColumn(col_name, F.col(col_name).cast("timestamp"))

            # Escribir en Snowflake
            table_name = f"{service.upper()}_TRIPS"

            # COLUMNA NUEVA DE ANIOS POSTERIORES
            if "CBD_CONGESTION_FEE" not in df.columns:
                # Si no está, agrega la columna con valor None (o un valor predeterminado)
                df = df.withColumn("CBD_CONGESTION_FEE", F.lit(None).cast(T.DoubleType()))

            # Eliminar registros previos de la misma ejecución (idempotencia)
            spark.read \
                .format("snowflake") \
                .options(**sfOptions) \
                .option("query", f"DELETE FROM {table_name} WHERE RUN_ID = '{run_id}'") \
                .load()
            
            # Reordenar columnas según ordered_cols para que no de error
            df = df.select([c for c in ordered_cols if c in df.columns])

                

            df.write \
                .format("snowflake") \
                .options(**sfOptions) \
                .option("dbtable", table_name) \
                .mode("append") \
                .save() 
            print(f"Datos de {service} para {year}-{month} cargados en Snowflake")

            # Eliminar archivo temporal
            os.remove(tmp_path) 




In [8]:
# cargar taxi zones
url_zones = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"
# Descargar archivo temporalmente
tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
tmp_path = tmp_file.name
tmp_file.close()
r = requests.get(url_zones)
with open(tmp_path, "wb") as f:
    f.write(r.content)
df_zones = spark.read.csv(tmp_path, header=True, inferSchema=True)
print(f"Archivo {url_zones} cargado en DataFrame")


df_zones.write \
    .format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "TAXI_ZONES") \
    .mode("overwrite") \
    .save()

Archivo https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv cargado en DataFrame
